# Searching for best hyperparameters set

In [188]:
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_parallel_coordinate
from itertools import chain, combinations
import plotly.express as px

In [189]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [190]:
train_df = pd.read_feather('./data/train_processed.ftr')
val_df = pd.read_feather('./data/val_processed.ftr')
cols = list(train_df.columns)

In [191]:
cols

['is_g734s',
 'CryoSleep',
 'VIP',
 'Europa',
 'Mars',
 'PSO J318.5-22',
 'TRAPPIST-1e',
 'Age',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'segment',
 'Transported']

## Prepare AdaBoost model

In [192]:
train_x = train_df[cols[:-1]].to_numpy()
train_y = train_df[cols[-1]].to_numpy()
val_x = val_df[cols[:-1]].to_numpy()
val_y = val_df[cols[-1]].to_numpy()

In [193]:
class ModelTester:
    
    def __init__(
        self,
        params: dict,
        train_x: np.array,
        train_y: np.array,
        val_x: np.array,
        val_y: np.array
    ):
        self.params = params
        self.train_x = train_x
        self.train_y = train_y
        self.val_x = val_x
        self.val_y = val_y
        
        self.model = self.train_model()
        self.y_pred_bin = self.get_bin_predictions()
            
    def train_model(self):
        # 1 - Run model
        tree = DecisionTreeClassifier(
            min_samples_split=params['min_samples_split'],
            max_depth=params['max_depth'],
            min_impurity_decrease=params['min_impurity_decrease'],
            criterion=params['criterion']
        )
        model = AdaBoostClassifier(
            base_estimator=tree,
            n_estimators=self.params['n_estimators'],
            learning_rate=self.params['learning_rate'],
            algorithm=self.params['algorithm'],
        )
        model.fit(self.train_x, self.train_y)
        return model
    
    def get_bin_predictions(self):
        # 2 - Get predictions
        y_pred = self.model.predict(val_x)
        y_pred_bin = [1 if y else 0 for y in y_pred]
        
        return y_pred_bin
    
    def get_accuracy(self):
        return round(accuracy_score(self.val_y, self.y_pred_bin), 3)
    
    def get_class_report(self):
        return classification_report(self.val_y, self.y_pred_bin)    

## Optuna research

In [194]:
def objective(trial):
    # 0 - Prepare params
    n_estimators = trial.suggest_int("n_estimators", 10, 200)
    algorithm = trial.suggest_categorical('algorithm', ['SAMME', 'SAMME.R'])
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.5)
    max_depth = trial.suggest_int('max_depth', 1, 100)
    min_samples_split = trial.suggest_int('min_samples_split', 1, 100)
    min_impurity_decrease = trial.suggest_float('min_impurity_decrease', 0, 0.0001)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy', 'logg_loss'])
    
    params = {
        'n_estimators': n_estimators,
        'learning_rate': learning_rate,
        'algorithm': algorithm,
        'max_depth': max_depth,
        'min_samples_split': min_samples_split,
        'min_impurity_decrease': min_impurity_decrease,
        'criterion': criterion
    }
    
    # 1 - Model
    model_tester = ModelTester(params, train_x, train_y, val_x, val_y)
    acc = model_tester.get_accuracy()
    error = 1 - acc
    
    return error

In [50]:
study = optuna.create_study()
study.optimize(objective, n_trials=1000)

[I 2022-07-01 14:07:06,516] A new study created in memory with name: no-name-2b4906b0-72d6-48c2-8cfd-3abae7f25359
[I 2022-07-01 14:07:06,810] Trial 0 finished with value: 0.20899999999999996 and parameters: {'n_estimators': 42, 'algorithm': 'SAMME.R', 'learning_rate': 0.3581669108603753, 'max_depth': 85, 'min_samples_split': 46, 'min_impurity_decrease': 9.809316965828954e-05, 'criterion': 'entropy'}. Best is trial 0 with value: 0.20899999999999996.
[I 2022-07-01 14:07:07,344] Trial 1 finished with value: 0.20099999999999996 and parameters: {'n_estimators': 82, 'algorithm': 'SAMME.R', 'learning_rate': 0.36698468988769967, 'max_depth': 7, 'min_samples_split': 50, 'min_impurity_decrease': 8.823835872333633e-05, 'criterion': 'entropy'}. Best is trial 1 with value: 0.20099999999999996.
[I 2022-07-01 14:07:07,576] Trial 2 finished with value: 0.21099999999999997 and parameters: {'n_estimators': 33, 'algorithm': 'SAMME.R', 'learning_rate': 0.4101814862741066, 'max_depth': 56, 'min_samples_spl

[I 2022-07-01 14:07:22,367] Trial 24 finished with value: 0.19999999999999996 and parameters: {'n_estimators': 87, 'algorithm': 'SAMME.R', 'learning_rate': 0.43217092378723465, 'max_depth': 14, 'min_samples_split': 9, 'min_impurity_decrease': 7.198408196599186e-05, 'criterion': 'entropy'}. Best is trial 17 with value: 0.19699999999999995.
[I 2022-07-01 14:07:22,951] Trial 25 finished with value: 0.21199999999999997 and parameters: {'n_estimators': 96, 'algorithm': 'SAMME.R', 'learning_rate': 0.11399082886209569, 'max_depth': 45, 'min_samples_split': 31, 'min_impurity_decrease': 8.392579230814408e-05, 'criterion': 'logg_loss'}. Best is trial 17 with value: 0.19699999999999995.
[I 2022-07-01 14:07:23,421] Trial 26 finished with value: 0.20399999999999996 and parameters: {'n_estimators': 71, 'algorithm': 'SAMME.R', 'learning_rate': 0.3283708601848096, 'max_depth': 65, 'min_samples_split': 64, 'min_impurity_decrease': 7.616875634458311e-05, 'criterion': 'entropy'}. Best is trial 17 with va

[I 2022-07-01 14:07:43,439] Trial 48 finished with value: 0.19699999999999995 and parameters: {'n_estimators': 181, 'algorithm': 'SAMME.R', 'learning_rate': 0.25820759308043173, 'max_depth': 94, 'min_samples_split': 6, 'min_impurity_decrease': 8.793869642780979e-05, 'criterion': 'gini'}. Best is trial 17 with value: 0.19699999999999995.
[I 2022-07-01 14:07:44,463] Trial 49 finished with value: 0.237 and parameters: {'n_estimators': 194, 'algorithm': 'SAMME', 'learning_rate': 0.2859607278647748, 'max_depth': 91, 'min_samples_split': 5, 'min_impurity_decrease': 9.109845587791916e-05, 'criterion': 'logg_loss'}. Best is trial 17 with value: 0.19699999999999995.
[I 2022-07-01 14:07:45,550] Trial 50 finished with value: 0.19899999999999995 and parameters: {'n_estimators': 182, 'algorithm': 'SAMME.R', 'learning_rate': 0.26146525285240907, 'max_depth': 98, 'min_samples_split': 5, 'min_impurity_decrease': 8.671615753452972e-05, 'criterion': 'gini'}. Best is trial 17 with value: 0.19699999999999

[I 2022-07-01 14:08:00,498] Trial 72 finished with value: 0.19499999999999995 and parameters: {'n_estimators': 110, 'algorithm': 'SAMME.R', 'learning_rate': 0.4767276789201188, 'max_depth': 19, 'min_samples_split': 36, 'min_impurity_decrease': 1.1613464732863478e-05, 'criterion': 'logg_loss'}. Best is trial 53 with value: 0.19099999999999995.
[I 2022-07-01 14:08:01,053] Trial 73 finished with value: 0.19599999999999995 and parameters: {'n_estimators': 90, 'algorithm': 'SAMME.R', 'learning_rate': 0.4744104420160877, 'max_depth': 23, 'min_samples_split': 42, 'min_impurity_decrease': 1.8886790664043046e-05, 'criterion': 'logg_loss'}. Best is trial 53 with value: 0.19099999999999995.
[I 2022-07-01 14:08:01,741] Trial 74 finished with value: 0.19599999999999995 and parameters: {'n_estimators': 102, 'algorithm': 'SAMME.R', 'learning_rate': 0.4846278230838988, 'max_depth': 18, 'min_samples_split': 42, 'min_impurity_decrease': 1.0830608560292116e-05, 'criterion': 'logg_loss'}. Best is trial 53

[I 2022-07-01 14:08:15,593] Trial 95 finished with value: 0.19699999999999995 and parameters: {'n_estimators': 93, 'algorithm': 'SAMME.R', 'learning_rate': 0.4402731273858903, 'max_depth': 21, 'min_samples_split': 39, 'min_impurity_decrease': 2.7819990776468042e-05, 'criterion': 'logg_loss'}. Best is trial 53 with value: 0.19099999999999995.
[I 2022-07-01 14:08:16,084] Trial 96 finished with value: 0.19999999999999996 and parameters: {'n_estimators': 78, 'algorithm': 'SAMME.R', 'learning_rate': 0.4469332813955577, 'max_depth': 19, 'min_samples_split': 40, 'min_impurity_decrease': 4.524030868889235e-06, 'criterion': 'logg_loss'}. Best is trial 53 with value: 0.19099999999999995.
[I 2022-07-01 14:08:16,758] Trial 97 finished with value: 0.19899999999999995 and parameters: {'n_estimators': 102, 'algorithm': 'SAMME.R', 'learning_rate': 0.4148584009558838, 'max_depth': 14, 'min_samples_split': 23, 'min_impurity_decrease': 3.093432139409756e-05, 'criterion': 'entropy'}. Best is trial 53 with

...

[I 2022-07-01 14:18:04,463] Trial 886 finished with value: 0.19299999999999995 and parameters: {'n_estimators': 99, 'algorithm': 'SAMME.R', 'learning_rate': 0.4861634898990018, 'max_depth': 3, 'min_samples_split': 55, 'min_impurity_decrease': 2.5247818796856896e-05, 'criterion': 'gini'}. Best is trial 53 with value: 0.19099999999999995.
[I 2022-07-01 14:18:05,318] Trial 887 finished with value: 0.19699999999999995 and parameters: {'n_estimators': 135, 'algorithm': 'SAMME.R', 'learning_rate': 0.4790543770317171, 'max_depth': 76, 'min_samples_split': 30, 'min_impurity_decrease': 2.117133667209157e-05, 'criterion': 'gini'}. Best is trial 53 with value: 0.19099999999999995.
[I 2022-07-01 14:18:05,891] Trial 888 finished with value: 0.238 and parameters: {'n_estimators': 94, 'algorithm': 'SAMME.R', 'learning_rate': 0.0703235402890802, 'max_depth': 3, 'min_samples_split': 68, 'min_impurity_decrease': 2.0220391948904366e-05, 'criterion': 'gini'}. Best is trial 53 with value: 0.190999999999999

[I 2022-07-01 14:18:21,784] Trial 910 finished with value: 0.19699999999999995 and parameters: {'n_estimators': 128, 'algorithm': 'SAMME.R', 'learning_rate': 0.4523638010530201, 'max_depth': 1, 'min_samples_split': 4, 'min_impurity_decrease': 9.273080307664667e-05, 'criterion': 'logg_loss'}. Best is trial 53 with value: 0.19099999999999995.
[I 2022-07-01 14:18:22,488] Trial 911 finished with value: 0.19699999999999995 and parameters: {'n_estimators': 109, 'algorithm': 'SAMME.R', 'learning_rate': 0.49954011986802654, 'max_depth': 12, 'min_samples_split': 67, 'min_impurity_decrease': 7.268230002912171e-05, 'criterion': 'logg_loss'}. Best is trial 53 with value: 0.19099999999999995.
[I 2022-07-01 14:18:23,290] Trial 912 finished with value: 0.19699999999999995 and parameters: {'n_estimators': 121, 'algorithm': 'SAMME.R', 'learning_rate': 0.48769418196087916, 'max_depth': 25, 'min_samples_split': 29, 'min_impurity_decrease': 6.77910982565568e-05, 'criterion': 'gini'}. Best is trial 53 with

[I 2022-07-01 14:18:39,610] Trial 934 finished with value: 0.19399999999999995 and parameters: {'n_estimators': 112, 'algorithm': 'SAMME.R', 'learning_rate': 0.4716855759227958, 'max_depth': 1, 'min_samples_split': 50, 'min_impurity_decrease': 8.470246688516253e-05, 'criterion': 'gini'}. Best is trial 53 with value: 0.19099999999999995.
[I 2022-07-01 14:18:40,403] Trial 935 finished with value: 0.19599999999999995 and parameters: {'n_estimators': 111, 'algorithm': 'SAMME.R', 'learning_rate': 0.4995927646664601, 'max_depth': 4, 'min_samples_split': 63, 'min_impurity_decrease': 8.213464086851208e-05, 'criterion': 'entropy'}. Best is trial 53 with value: 0.19099999999999995.
[I 2022-07-01 14:18:41,263] Trial 936 finished with value: 0.19099999999999995 and parameters: {'n_estimators': 129, 'algorithm': 'SAMME.R', 'learning_rate': 0.48131923356527856, 'max_depth': 6, 'min_samples_split': 87, 'min_impurity_decrease': 9.305015883207253e-05, 'criterion': 'gini'}. Best is trial 53 with value: 

[I 2022-07-01 14:18:58,847] Trial 958 finished with value: 0.235 and parameters: {'n_estimators': 126, 'algorithm': 'SAMME', 'learning_rate': 0.4098195620164426, 'max_depth': 49, 'min_samples_split': 82, 'min_impurity_decrease': 2.3054043326608573e-05, 'criterion': 'gini'}. Best is trial 53 with value: 0.19099999999999995.
[I 2022-07-01 14:18:59,560] Trial 959 finished with value: 0.19699999999999995 and parameters: {'n_estimators': 106, 'algorithm': 'SAMME.R', 'learning_rate': 0.4993430968951064, 'max_depth': 65, 'min_samples_split': 97, 'min_impurity_decrease': 9.077324722768247e-05, 'criterion': 'logg_loss'}. Best is trial 53 with value: 0.19099999999999995.
[I 2022-07-01 14:19:00,316] Trial 960 finished with value: 0.19399999999999995 and parameters: {'n_estimators': 103, 'algorithm': 'SAMME.R', 'learning_rate': 0.4912557320813694, 'max_depth': 3, 'min_samples_split': 62, 'min_impurity_decrease': 1.7084737424692996e-05, 'criterion': 'entropy'}. Best is trial 53 with value: 0.190999

[I 2022-07-01 14:19:17,332] Trial 982 finished with value: 0.19199999999999995 and parameters: {'n_estimators': 126, 'algorithm': 'SAMME.R', 'learning_rate': 0.4704946568276425, 'max_depth': 50, 'min_samples_split': 63, 'min_impurity_decrease': 5.799538796748643e-05, 'criterion': 'logg_loss'}. Best is trial 53 with value: 0.19099999999999995.
[I 2022-07-01 14:19:18,282] Trial 983 finished with value: 0.19699999999999995 and parameters: {'n_estimators': 134, 'algorithm': 'SAMME.R', 'learning_rate': 0.48647724575246387, 'max_depth': 67, 'min_samples_split': 84, 'min_impurity_decrease': 9.235101988081695e-05, 'criterion': 'gini'}. Best is trial 53 with value: 0.19099999999999995.
[I 2022-07-01 14:19:19,233] Trial 984 finished with value: 0.19999999999999996 and parameters: {'n_estimators': 132, 'algorithm': 'SAMME.R', 'learning_rate': 0.46534435240365885, 'max_depth': 64, 'min_samples_split': 90, 'min_impurity_decrease': 2.4817602316813385e-05, 'criterion': 'gini'}. Best is trial 53 with 

### Show results

In [55]:
plot_optimization_history(study)

In [56]:
plot_param_importances(study)

In [57]:
key_params = ['algorithm', 'learning_rate', 'n_estimators']
plot_slice(study, params = key_params).show()

In [195]:
study.best_params

{'n_estimators': 106,
 'algorithm': 'SAMME.R',
 'learning_rate': 0.4827054897488229,
 'max_depth': 11,
 'min_samples_split': 1,
 'min_impurity_decrease': 7.200517833654635e-05,
 'criterion': 'gini'}

In [196]:
best_params = {
    'n_estimators': 106,
    'algorithm': 'SAMME.R',
    'learning_rate': 0.4827054897488229,
    'max_depth': 11,
    'min_samples_split': 1,
    'min_impurity_decrease': 7.200517833654635e-05,
    'criterion': 'gini'
}

## Create model with best params

In [197]:
train_x = train_df[cols[:-1]].to_numpy()
train_y = train_df[cols[-1]].to_numpy()
val_x = val_df[cols[:-1]].to_numpy()
val_y = val_df[cols[-1]].to_numpy()

In [198]:
params = best_params
tree = DecisionTreeClassifier(
#     min_samples_split=1,
    max_depth=11,
    min_impurity_decrease=7.200517833654635e-05,
    criterion='gini'
)
model = AdaBoostClassifier(
    base_estimator=tree,
    n_estimators=106,
    learning_rate=0.4827054897488229,
    algorithm='SAMME.R'
)

In [199]:
X = pd.concat([train_df[cols[:-1]], val_df[cols[:-1]]])
y = pd.concat([train_df[cols[-1]], val_df[cols[-1]]])

In [200]:
cv_scores = cross_val_score(model, X, y, cv=10)
print(f'Mean cv (k=10) accuracy score = {round(np.mean(cv_scores), 3)}')

Mean cv (k=10) accuracy score = 0.751


## Feature selection

In [201]:
model.fit(train_x, train_y)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=11,
                                                         min_impurity_decrease=7.200517833654635e-05),
                   learning_rate=0.4827054897488229, n_estimators=106)

In [203]:
imp_df = pd.DataFrame({
    'feature': cols[:-1],
    'imp': model.feature_importances_
}).sort_values('imp', ascending=False)
best_features = imp_df['feature'].to_list()
imp_df

,feature,imp
7,Age,0.026601
9,FoodCourt,0.019832
10,ShoppingMall,0.017611
8,RoomService,0.015871
12,VRDeck,0.015086
11,Spa,0.014537
1,CryoSleep,0.008980
4,Mars,0.002921
5,PSO J318.5-22,0.002487
3,Europa,0.002379


In [204]:
temp_cols = []
cv_list = []

for feature in best_features:
    temp_cols.append(feature)
    X = pd.concat([train_df[temp_cols], val_df[temp_cols]]).to_numpy()
    cv_score = cross_val_score(model, X, y, cv=10)
    cv_list.append(cv_score)

In [212]:
means = []
for scores in cv_list:
    means.append(round(np.mean(scores), 3))

i_features = 0
for i in range(len(means)):
    if means[i] == max(means):
        i_features = i
        print(i_features)
        break

7


In [215]:
best_features[:i_features+1]

['Age',
 'FoodCourt',
 'ShoppingMall',
 'RoomService',
 'VRDeck',
 'Spa',
 'CryoSleep',
 'Mars']

## Make final model

In [216]:
train_x = train_df[best_features[:i_features+1]].to_numpy()
val_x = val_df[best_features[:i_features+1]].to_numpy()

In [224]:
params = best_params
tree = DecisionTreeClassifier(
#     min_samples_split=1,
    max_depth=11,
    min_impurity_decrease=7.200517833654635e-05,
    criterion='gini'
)
model = AdaBoostClassifier(
    base_estimator=tree,
    n_estimators=106,
    learning_rate=0.4827054897488229,
    algorithm='SAMME.R'
)

In [225]:
model.fit(train_x, train_y)
pred_val = model.predict_proba(val_x)
pred_train = model.predict_proba(train_x)

In [238]:
train_proba_df = pd.DataFrame(pred_train)
train_proba_df.columns = ['0', 'ada_boost']
train_proba_df['y'] = train_y.astype(int)
train_proba_df.drop(columns='0', inplace=True)
train_proba_df

,ada_boost,y
0,0.507872,0
1,0.505969,0
2,0.366664,0
3,0.501475,1
4,0.294826,0
...,...,...
6996,0.401386,0
6997,0.247687,0
6998,0.542936,1
6999,0.455905,0


In [237]:
val_proba_df = pd.DataFrame(pred_val)
val_proba_df.columns = ['0', 'ada_boost']
val_proba_df['y'] = val_y.astype(int)
val_proba_df.drop(columns='0', inplace=True)
val_proba_df

,ada_boost,y
0,0.564468,1
1,0.503813,0
2,0.437986,1
3,0.434112,0
4,0.504742,1
...,...,...
1687,0.379731,0
1688,0.507257,1
1689,0.509272,1
1690,0.493184,1


In [239]:
train_proba_df.to_csv('./data/ensemble_train_df.csv')
val_proba_df.to_csv('./data/ensemble_val_df.csv')